<a href="https://colab.research.google.com/github/drawnator/PLN-grupo-19/blob/main/Grupo19_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ferramenta otimizadora de aleatoriedade em senhas
---
Modelo de linguagem que auxilia a tornar senhas mais fortes, analisando a entrada e dando sugestões que tornariam a senha mais improvável de adivinhar.

Assuntos:
- analise de frequencia
- masked language model

técnologia:
- bert
- RNN
- arvore de decisão
- senha aleatória

In [ ]:
import requests
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, SimpleRNN
from keras.optimizers import Adam

# masked language mode

## read and split dataframe

In [ ]:
# RECOMENDO FORTEMENTE BAIXAR E ARRASTAR MANUALMENTE ATÉ ARTIGOS, ISSO AQUI DEMORA MT \/
url = "https://github.com/brannondorsey/naive-hashcat/releases/download/data/rockyou.txt"
# response = requests.get(url)
# data = response.text
#...

In [ ]:
with open("rockyou.txt", "r", encoding='latin-1') as f:
  df = pd.DataFrame(f.readlines(), columns=['password'])

In [ ]:
df['password'] = df['password'].str.replace('\n', '')

In [ ]:
# if not os.path.exists("rockyou.csv"):
#   df.to_csv("rockyou.csv")

In [ ]:
df.shape

(14344391, 1)

## tokenization

In [ ]:
#https://colab.research.google.com/drive/1Suv_JhRhoYNOCHtrGQwqZQO18nMXHEfq?usp=sharing
def mask_tokens(inputs, tokenizer, mlm_probability=0.10):
    inputs = np.array(inputs)
    labels = np.copy(inputs)

    rand = np.random.rand(*inputs.shape)
    mask_arr = (rand < mlm_probability)

    special_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id]
    for special_id in special_tokens:
        mask_arr[inputs == special_id] = False

    inputs[mask_arr] = tokenizer.mask_token_id

    labels[~mask_arr] = 0

    return inputs, labels

In [ ]:
all_chars = sorted(list(set(''.join(df['password']))))
char_to_int = {char: i for i, char in enumerate(all_chars)}
int_to_char = {i: char for char, i in char_to_int.items()}

In [ ]:
class DummyTokenizer:
    def __init__(self, char_to_int,int_to_char,max_length = 32):
        self.char_to_int = char_to_int
        self.int_to_char = int_to_char
        self.mask_token_id = char_to_int['[MASK]']
        self.cls_token_id = char_to_int['[CLS]']
        self.sep_token_id = char_to_int['[SEP]']
        self.pad_token_id = char_to_int['[PAD]']
        self.max_length = max_length
        self.vocab_size = len(char_to_int)

    def character_tokenizer(self,text):
      token = []
      for i in range(self.max_length):
        if (i < len(text)):
          token.append(char_to_int[text[i]])
        else:
          token.append(char_to_int['[PAD]'])
      return token

In [ ]:
char_to_int['[MASK]'] = len(char_to_int)
char_to_int['[CLS]'] = len(char_to_int)
char_to_int['[SEP]'] = len(char_to_int)
char_to_int['[PAD]'] = len(char_to_int)
int_to_char = {i: char for char, i in char_to_int.items()}

In [ ]:
dummy_tokenizer = DummyTokenizer(char_to_int,int_to_char)

In [ ]:
df['tokenized_password'] = df['password'].apply(dummy_tokenizer.character_tokenizer)

In [ ]:
masked_inputs = []
masked_labels = []

In [ ]:
for tokenized_list in df['tokenized_password'].head().tolist():
    inputs, labels = mask_tokens(tokenized_list, dummy_tokenizer)
    masked_inputs.append(inputs)
    masked_labels.append(labels)

NameError: name 'mask_tokens' is not defined

## Splitting data

In [ ]:
train, temp = train_test_split(df, test_size=0.2)
val, test = train_test_split(df, test_size=0.5)

In [ ]:
def create_train_val_test_datasets(df, tokenizer, test_size=0.10, val_size=0.15, batch_size=8,max=None):

    if max:
        df = df.head(max)

    train_df, temp_df = train_test_split(df, test_size=test_size, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=val_size, random_state=42)

    def df_to_dataset(dataframe, tokenizer, batch_size):
        tokenized_passwords = dataframe['tokenized_password'].tolist()

        masked_inputs = []
        masked_labels = []
        for tokens in tokenized_passwords:
            inputs, labels = mask_tokens(tokens, tokenizer)
            masked_inputs.append(inputs)
            masked_labels.append(labels)

        input_ids = tf.constant(masked_inputs, dtype=tf.int32)
        labels = tf.constant(masked_labels, dtype=tf.int32)

        dataset = tf.data.Dataset.from_tensor_slices((input_ids, labels))
        return dataset.shuffle(1000).batch(batch_size)

    train_dataset = df_to_dataset(train_df, tokenizer, batch_size)
    val_dataset = df_to_dataset(val_df, tokenizer, batch_size)
    test_dataset = df_to_dataset(test_df, tokenizer, batch_size)

    return train_dataset, val_dataset, test_dataset

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

def create_train_val_test_arrays(df, tokenizer, test_size=0.2, val_size=0.5, max_length=32):
    train_df, temp_df = train_test_split(df, test_size=test_size, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=val_size, random_state=42)

    def dataframe_to_arrays(dataframe, tokenizer, max_length):
        tokenized_passwords = dataframe['tokenized_password'].tolist()

        masked_inputs = []
        masked_labels = []

        for tokens in tokenized_passwords:
          for i in range(1,len(tokens)):
            input_seq = tokens[:i]
            label = tokens[i]
            input_seq += [tokenizer.pad_token_id] * (max_length - len(input_seq))
            masked_inputs.append(input_seq)
            masked_labels.append(label)

        return np.array(masked_inputs), np.array(masked_labels)

    train_inputs, train_labels = dataframe_to_arrays(train_df, tokenizer, max_length)
    val_inputs, val_labels = dataframe_to_arrays(val_df, tokenizer, max_length)
    test_inputs, test_labels = dataframe_to_arrays(test_df, tokenizer, max_length)

    return train_inputs, train_labels, val_inputs, val_labels, test_inputs, test_labels

In [ ]:
train_ds, val_ds, test_ds = create_train_val_test_datasets(df, dummy_tokenizer, batch_size=8)

NameError: name 'create_train_val_test_datasets' is not defined

In [ ]:
train_inputs, train_labels, val_inputs, val_labels, test_inputs, test_labels = create_train_val_test_arrays(df, dummy_tokenizer)

# defining a model
TODO: modelo RNN mas train label usados no treinamento do bert, opções:
- mudar modelo a baixo para fazer fine tunning do bert
- mudar mask_tokens para gerar uma entrada e saida condizente com um problema RNN


In [ ]:
#https://colab.research.google.com/drive/1mts5E3yAd1irLzS7Ei6UtwbG773C87DB?usp=sharing
model = Sequential([
    Embedding(
        input_dim=dummy_tokenizer.vocab_size,
        output_dim=100,
        input_shape=(dummy_tokenizer.max_length,)),
    SimpleRNN(64, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(dummy_tokenizer.vocab_size, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    train_inputs,
    train_labels,
    validation_data=(val_inputs, val_labels),
    epochs=3,
    batch_size=256,
    verbose=1)